최근 자연어 처리 커뮤니티에서는 트랜스포머 기반의 여러 사전 학습모델을 묶어 만든 앙상블(ensemble) 모델이

뛰어난 성능을 보여준다.

질의응답 직업 벤치마크 결과를 섞어 만든 SQuAD2.0에 최고 성능이 모델 10개가 모두 앙상블 모델일 정도입니다.

In [2]:
datasets = [ 
    ['What music do you like?', 'I like Rock music.', 1],
    ['What is your favorite food?', 'I like sushi the best', 1],
    ['What is your favorite color?', "I'm going to be a doctor", 0],
    ['What is your favorite song?', "Tokyo olympic game in 2020 was postponed", 0],
    ['Do you like watching TV shows?', "Yeah, I often watch it in my spear time", 1]
]

In [ ]:
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AdamW
from torch import nn
import torch

class BertEnsembleForNextSentencePrediction(BertPreTrainedModel):
    """
    BERT 앙상블 모델을 사용한 Next Sentence Prediction(NSP).
    동일한 BERT 모델을 두 개 사용하여 서로 다른 입력을 처리한 후,
    두 개의 출력 벡터를 연결 (concatenation)하여 최종 예측을 수행.
    """
    def __init__(self, config, *args, **kwargs):
        super().__init__(config)

        # 두 개의 동일한 BERT 모델을 초기화 (독립적인 가중치 설정)
        self.bert_model_1 = BertModel(config)
        self.bert_model_2 = BertModel(config)

        # 두 BERT 모델의 출력(hidden state)을 겨합한 후, 이진 분류를 수행  output_layer = 2
        self.cls = nn.Linear(2 * self.config.hidden_size, 2)

        # BERT 모델의 가중치를 초기화.
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        next_sentence_label=None
    ):
        """
        Foward 함수: 두 개의 BERT 모델을 사용하여 입력을 처리한 후, NSP예측 수행.
        """
        outputs = []
        # 첫 번째 모델에 입력할 토큰 ID 및 어텐션 마스크 추출
        input_ids_1 = input_ids[0]
        attention_mask_1 = attention_mask[0]

        # 첫 번째 BERT 모델을 통해 hidden state(input_ids) 출력 (pooler output 사용)
        outputs.append(self.bert_model_1(input_ids_1, attention_mask=attention_mask_1))

        # 두 번째 BERT 모델에 입력할 토큰 ID 및 어텐션 마스크 추출.
        input_ids_2 = input_ids[1]
        attention_mask_2 = attention_mask[1]

        # 두 번째 BERT 모델을 통해 hidden state(input_ids) 출력 (pooler output 사용)
        outputs.append(self.bert_model_2(input_ids_2, attention_mask=attention_mask_2))

        # 두 BERT 모델의 출력(hidden state)을 연결(concatenation)
        # 두 개의 BERT 모델에서 얻은 pooler output을 연결하여 특정 백터 구현
        last_hidden_states = torch.cat([output[1] for output in outputs], dim=1)

        # 최종적으로 연결된 특징 벡터를 이진 분류 레이어에 전달하여 NSP 예측.
        # self.cls 선형함수에 마지막 은닉층 임베딩 상태를 투입하여 로짓 추출.
        logits = self.cls(last_hidden_states)

        # 크로스엔트로피 손실 (crossentropy loss) 구하기 (손실율 정답과 멀어질 수 록 가중치 부여.)
        if next_sentence_label is not None:
            # CrossEntropyLoss를 사용하여 NSP 라벨에 대한 손실 계산
            loss_fct = nn.CrossEntropyLoss(ignore_index=-1)

            next_sentence_label = loss_fct(logits.view(-1, 2), next_sentence_label.view(-1))
            return next_sentence_label, logits
        else:
            return logits

/mnt/d/workspace/colab_try/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 앙상블 트레이닝에 사용할 사전학습 BERT 불러오기
# BERT 앙상블 학습 클래스를 인스턴스화하고 이를 GPU에 전달하세요. 아울러
# Optimizer 변수에 최적화 함수로 AmdmW를 대입하세요 가중치 감소 가능을 통해 과적합을 방지합니다.
import torch
from transformers import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 및 Config 설정
config = BertConfig()
model = BertEnsembleForNextSentencePrediction(config)

# 토크나이저 설정
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# 학습률 설정
learning_rate = 1e-5

# 절편과 가중치를 설정.
no_decay = ["bias", "LayerNorm.weight"]

# 최적화 함수 그룹 파라미터 설정
optimizer_grouped_parameters = [{
    "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
}]
# 최적화 함수 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

/mnt/d/workspace/colab_try/.venv/lib/python3.10/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 060 BERT 앙상블 학습 - 데이터 증강
def prepare_data(dataset, qa=True):
    """
    BERT 학습을 위한 데이터 전처리 및 증강 함수.

    - qa True -> 질문 Q -응답 A 순서로 변환
    - qa=False -> 응답A 질분 Q 순서로 데이터 변환 (데이터 증강.)
    Args:
        dataset (list): [(질문, 답변, 라벨), (질문, 답변, 라벨), ...] 형태의 데이터셋
        qa (bool): 순서

    Returns:
        input_ids(Tensor): 토큰화 된 입력 ID (BERT 모델 입력)
        attention_masks(Tensor): 어텐션 마스크 (패딩된 부분을 무시하도록 설정.)
        labels(List): 정답 라벨 리스트
    """    
    input_ids, attention_masks = [], []
    labels = []

    for point in dataset:
        if qa is True:
            # 데이터셋에서 질문 Q 응답 A 추출.
            q, a, _ = point
        else:
            # 응답A 질문Q 순서를 변경하여 데이터 증강.
            a, q, _ = point
        # BERT 토크나이저를 사용하여 입력 데이터 변환
        encoded_dict = tokenizer.encode_plus(
            q,  # 첫 번째 문장 (질문 or 응답)
            a,  # 두 번째 문장 (응답. or 질문)
            add_special_tokens=True,  # [CLS], [SEP] 토큰 추가
            max_length=128,  # 최대 길이 설정
            pad_to_max_length=True,  # 길이가 부족한 경우 패딩 추가
            return_attention_mask=True,  # 어텐션 마스크 반환 (패딩된 부분 무시)
            return_tensors='pt',  # PyTorch 텐서로 반환
            truncation=True  # 최대 길이를 초과하는 경우 잘라내기
        )
        # 변환된 토큰 ID와 어텐션 마스크 저장.
        input_ids.append(encoded_dict["input_ids"])
        attention_masks.append(encoded_dict["attention_mask"])
        # 데이터셋의 마지막 요소(정답 라벨) 저장
        labels.append(point[-1])
    # 리스트 형태의 텐서들을 하나의 텐서로 병합
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks, labels

In [ ]:
# 061 BERT앙상블 학습 커스텀 데이터 세트 정의
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, Dataset, SequentialSampler

# QADatasets 클래스 생성.
class QADataset(Dataset):
    """
    PyTorch Dataset을 상속받아 BERT 학습을 위한 데이터셋 클래스 정의

    Args:
        input_ids(Tensor): BERT 입력 토큰 ID(질문 - 응답 쌍의 토큰화 결과)
        attention_masks(Tensor): 어텐션 마스크 (패딩된 부분 무시)
        labels(Tensor, optional): 정답 라벨 (NSP 학습을 위한 레이블)
    """
    # input_ids 텐서와 attention_masks 텐서 생성
    def __init__(self, input_ids, attention_masks, labels=None):
        # Numpy 배열로 변환하여 저장 (메모리 최적화)
        self.input_ids = np.array(input_ids)
        self.attention_mask = np.array(attention_masks)
        # 정답 라벨을 Pytorch 텐서 타입으로 변환 
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.attention_mask[index], self.labels[index]

    def __len__(self):
        return self.input_ids.shape[0]

In [ ]:
# BERT 앙상블 학습d을 위한 데이터 로더

# prepare_data 함수 호출을 통해 데이터 전처리 수행
# 질문 Q 응답 A 순서로 데이터 변환 (기본 설정)
input_ids_qa, attention_masks_qa, labels_qa = prepare_data(datasets)

# 변환된 데이터를 커스텀 QADataset 클래스에 전달하여 dataset 객체 생성
train_dataset_qa = QADataset(input_ids_qa, attention_masks_qa, labels_qa)

# 응답 A 질문 Q 순서로 데이터를 변환하여 데이터셋 객체 생성.
input_ids_aq, attention_masks_aq, labels_aq = prepare_data(datasets, qa=False)

train_dataset_aq = QADataset(input_ids_aq, attention_masks_aq, labels_aq)

# 질문 Q -> 응답 A 데이터 로더 
dataloader_qa = DataLoader(
    dataset=train_dataset_qa,
    batch_size=5,
    sampler=SequentialSampler(train_dataset_qa)
)

dataloader_aq = DataLoader(
    dataset=train_dataset_aq,
    batch_size=5,
    sampler=SequentialSampler(train_dataset_aq)
)

/tmp/ipykernel_47482/2364885428.py:10: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.input_ids = np.array(input_ids)
/tmp/ipykernel_47482/2364885428.py:11: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.attention_mask = np.array(attention_masks)


In [ ]:
# BERT 앙상블 학습 파인튜닝.

from tqdm import tqdm

epochs = 30

progress = tqdm(range(epochs))

for epoch in progress:
    # 0, (qa, aq) 
    for step, combine_batch in enumerate(zip(dataloader_qa, dataloader_aq)):
        # QA AQ 배치를 통해 가져옴 
        batch_1, batch_2 = combine_batch
        # 모델을 학습모드로 설정.
        model.train()
        # 데이타를 GPU 또는 CPU로 이동
        batch_1 = tuple(t.to(device) for t in batch_1)
        batch_2 = tuple(t.to(device) for t in batch_2)

        # 모델 입력ㄱ밧 구성 (BERT 앙상블)
        inputs = {
            "input_ids": [batch_1[0], batch_2[0]],
            "attention_mask": [batch_1[1], batch_2[1]],
            "next_sentence_label": batch_1[2]
        }
        # Foward 수행 loss+logits
        outputs = model(**inputs)
        # 손실 (loss) 값 추출
        loss = outputs[0]
        # 역전파 수행( 손실값 기준으로 가중치 업데이트)
        loss.backward()
        # tqdm 진행바에 현재 에포크 및 손실값 출력
        progress.set_postfix_str(f"epoch: {epoch}, loss: {loss}")
        # 옵티마이저 가중치 업데이트
        optimizer.step()
        # 그래디언트 초가화.
        model.zero_grad()

100%|██████████| 30/30 [00:02<00:00, 11.19it/s, epoch: 29, loss: 0.00048139350838027894]


In [ ]:
# 테스트 데이터 Preprocessing
input_ids_qa, attention_masks_qa, labels_qa = prepare_data(datasets)

test_dataset_qa = QADataset(input_ids_qa, attention_masks_qa, labels_qa)

input_ids_aq, attention_masks_aq, labels_aq = prepare_data(datasets, qa=False)

test_dataset_aq = QADataset(input_ids_aq, attention_masks_aq, labels_aq)

dataloader_qa = DataLoader(
    dataset=test_dataset_qa,
    batch_size=16,
    sampler=SequentialSampler(test_dataset_qa)
)


dataloader_aq = DataLoader(
    dataset=test_dataset_aq,
    batch_size=16,
    sampler=SequentialSampler(test_dataset_aq)
)

complete_outputs, complete_label_ids = [], []

for step, combine_batch in enumerate(zip(dataloader_qa, dataloader_aq)):
    model.eval()

    batch_1, batch_2 = combine_batch

    batch_1 = tuple(t.to(device) for t in batch_1)
    batch_2 = tuple(t.to(device) for t in batch_2)

    with torch.no_grad():
        inputs = {
            "input_ids": [batch_1[0], batch_2[0]],
            "attention_mask": [batch_1[1], batch_2[1]],
            "next_sentence_label": batch_1[2]
        }

        outputs = model(**inputs)

        tmp_eval_loss, logits = outputs[:2]

        logits = logits.detach().cpu().numpy()

        outputs = np.argmax(logits, axis=1)

        labels_ids = inputs['next_sentence_label'].detach().cpu().numpy()

    complete_outputs.extend(outputs)
    complete_label_ids.extend(labels_ids)
    
print(complete_outputs, complete_label_ids)

/tmp/ipykernel_47482/2364885428.py:10: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.input_ids = np.array(input_ids)
/tmp/ipykernel_47482/2364885428.py:11: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.attention_mask = np.array(attention_masks)


[np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1)] [np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1)]


In [33]:
datasets = [["what music do you like?", "I like Rock Music", 1]]

input_ids_qa, attention_masks_qa, labels_qa = prepare_data(datasets)

test_dataset_qa = QADataset(input_ids_qa, attention_masks_qa, labels_qa)

input_ids_aq, attention_masks_aq, labels_aq = prepare_data(datasets, qa=False)

test_dataset_aq = QADataset(input_ids_aq, attention_masks_aq, labels_aq)

dataloader_qa = DataLoader(
    dataset=test_dataset_qa,
    batch_size=16,
    sampler=SequentialSampler(test_dataset_qa)
)


dataloader_aq = DataLoader(
    dataset=test_dataset_aq,
    batch_size=16,
    sampler=SequentialSampler(test_dataset_aq)
)

complete_outputs, complete_label_ids = [], []

for step, combine_batch in enumerate(zip(dataloader_qa, dataloader_aq)):
    model.eval()

    batch_1, batch_2 = combine_batch

    batch_1 = tuple(t.to(device) for t in batch_1)
    batch_2 = tuple(t.to(device) for t in batch_2)

    with torch.no_grad():
        inputs = {
            "input_ids": [batch_1[0], batch_2[0]],
            "attention_mask": [batch_1[1], batch_2[1]],
            "next_sentence_label": batch_1[2]
        }

        outputs = model(**inputs)

        tmp_eval_loss, logits = outputs[:2]

        logits = logits.detach().cpu().numpy()

        outputs = np.argmax(logits, axis=1)

        labels_ids = inputs['next_sentence_label'].detach().cpu().numpy()

    complete_outputs.extend(outputs)
    complete_label_ids.extend(labels_ids)

print(complete_outputs, complete_label_ids)

/mnt/d/workspace/colab_try/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2690: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/tmp/ipykernel_47482/2364885428.py:10: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.input_ids = np.array(input_ids)
/tmp/ipykernel_47482/2364885428.py:11: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword argume

[np.int64(1)] [np.int64(1)]
